# Detailed walk through the Airbnb Dataset

In [28]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series, DataFrame

import csv
import numpy as np
import seaborn as sns
from airbnb_tools import * 

# Machine Learning Packages
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler


# Setup plot environment
%matplotlib inline
sns.set_style("whitegrid")

In [2]:
train = pd.read_csv("data/train_users_2.csv")

In [4]:
train.columns

Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser', 'country_destination'],
      dtype='object')

In [53]:
keep_col = ["country_destination", # The output, need to keep this
            "gender", 
            "language", 
            "age"]

# Create X and trim out data that has NFD as the country, since they didn't book
X = pd.DataFrame()
for cols in keep_col:
    X[cols] = train[cols]

# Take out non-bookings
X = X[X["country_destination"] != "NDF"]


# For now, drop things with NA
X.dropna(axis=0, inplace=True)

#Now, split the categorical variables
cat_var = ["gender", "language"]
X = split_categorical_variables(X, cat_var)

# Setup array of outputs.
yFull = pd.get_dummies(X.country_destination)
X.drop(["country_destination"], axis=1, inplace=True)
y = yFull["US"]

from sklearn.preprocessing import StandardScaler

# Scale our features
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [33]:
# build test and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [34]:
# This is a function that returns a base input
def base_rate_model(X):
    y = np.zeros(X.shape[0])
    return y

In [36]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty="l2", C=1)

In [37]:
model.fit(X_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [47]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Base Model
base_roc_auc = roc_auc_score(y_test, base_rate_model(X_test))
print("Base Rate AUC = %2.2f" % base_roc_auc)
print(classification_report(y_test, base_rate_model(X_test)))

# Log Reg model
logit_roc_auc = roc_auc_score(y_test, model.predict(X_test))
print("Logistic AUC = %2.2f" % logit_roc_auc)
print(classification_report(y_test, model.predict(X_test)))

Base Rate AUC = 0.50
             precision    recall  f1-score   support

        0.0       0.29      1.00      0.45      3953
        1.0       0.00      0.00      0.00      9754

avg / total       0.08      0.29      0.13     13707

Logistic AUC = 0.50
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      3953
        1.0       0.71      1.00      0.83      9754

avg / total       0.51      0.71      0.59     13707



/Users/rcool/miniconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


AttributeError: 'LogisticRegression' object has no attribute 'feature_importance_'